In [175]:
#Import the libraries

import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta, date
import re

In [158]:
#Get the webpage and parse it as html

page = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.777120000000025&lon=-122.41963999999996#.YFs1e6_7QdW")
soup = BeautifulSoup(page.content, 'html.parser')
html = list(soup.children)[2]
body = list(html.children)[3]        # Required data is nested here

In [159]:
#Retrieve the day names nested in body 

day_names = body.find_all('div', class_ = 'col-sm-2 forecast-label') #day names are in this tag
list_of_days = []

for each_day in day_names:
    e = each_day.text                #'.text' returns the text inside the Tag
    list_of_days.append(e)

dict['Day'] = list_of_days
#dict.to_csv('')                     #tansform to csv so that others can import and work on the file

In [199]:
#Retrieve the weather summary nested inside the body and extract the necessary data

day_summary = body.find_all('div', class_ = 'col-sm-10 forecast-text') #day summary is in this tag

summary_list = []
for summary in day_summary:
    e = summary.text              
    summary_list.append(e)
print(summary_list)

summary_string = ' '.join(summary_list) #.split('.')  #convert list to string to retrieve other information

temp = re.findall("[0-9]\d[.]", summary_string) #regex searches for eg. '40.' pattern
dict['Temperature'] = temp
df = pd.DataFrame(dict)







['Partly cloudy, with a low around 48. West northwest wind 14 to 18 mph, with gusts as high as 32 mph. ', 'Sunny, with a high near 62. West wind 6 to 16 mph, with gusts as high as 26 mph. ', 'Clear, with a low around 46. West wind 9 to 14 mph becoming light west northwest. Winds could gust as high as 21 mph. ', 'Sunny, with a high near 68. North wind around 8 mph. ', 'Clear, with a low around 47. West wind 5 to 8 mph becoming calm  in the evening. ', 'Sunny, with a high near 70.', 'Clear, with a low around 48.', 'Sunny, with a high near 68.', 'Mostly clear, with a low around 48.', 'Sunny, with a high near 64.', 'Clear, with a low around 47.', 'Sunny, with a high near 69.', 'Clear, with a low around 49.', 'Sunny, with a high near 72.']


In [196]:
#create a dynamic date format

todays_date = date.today()
final_date = todays_date + timedelta(days=6)
delta = todays_date - final_date          # as timedelta
date_range = pd.date_range(start=todays_date, end=final_date) #for a week
date_range = date_range.repeat(2) #Each day has morning and evening
print(date_range)

DatetimeIndex(['2021-03-25', '2021-03-25', '2021-03-26', '2021-03-26',
               '2021-03-27', '2021-03-27', '2021-03-28', '2021-03-28',
               '2021-03-29', '2021-03-29', '2021-03-30', '2021-03-30',
               '2021-03-31', '2021-03-31'],
              dtype='datetime64[ns]', freq=None)


In [ ]:
#Convert Fahrenheit to celcius 
def celcius(f):
    return int(f*(5/9))

In [ ]:
#for day in day_container:
 #   x = day_container.find('p', class_ ='period-name')
  #  my_dummy_list.append(x)
#print(my_dummy_list)
x = day_container.find_all('p', class_ ='period-name')
#print(x)

s = str(x[0])
s = s.split("<br/><br/>")

In [ ]:



period_name = body.find('p', class_ = 'period-name')
period_name.text



In [ ]:
#body1 = list(body.children)[1]

#print(body1)


In [ ]:
#target parents
#go deeper into childen
#find p element or relevant class
#target 7
#target temperature
#target summary
#save into a list
#save into a dictionary?

In [ ]:
#{"day1":["sunday", "54", 'today was a great sunny day '], "day2":["monday", "54", 'today was a great sunny day ']}

In [ ]:
#load dict into pandas
# create column names if needed

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4b76008c-93f2-488a-b8d9-c179cb7406d1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>